# Movies and Books Assistant with Weaviate Query Agent
**Demo for the Reimanining Data Workflows Webinar - April 29th 2025**

Example code for how we can build an agent which can answer quuestions about books and movies, using the [Weaviate Query Agent](https://weaviate.io/developers/agents?utm_source=webinar&utm_campaign=agents&utm_content=reimagining-data-workflows).

> 📚 You can read and learn more about this service in our ["Introducing the Weaviate Query Agent"](https://weaviate.io/blog/query-agent?utm_source=webinar&utm_campaign=agents&utm_content=reimagining-data-workflows) blog.

To get started, we've prepared a few open datasets, available on Hugging Face. The first step will be walking through how to populate your Weaviate Cloud collections.

- [**Movies:**](https://huggingface.co/datasets/weaviate/agents/viewer/personalization-agent-movies) A dataset that lists movies, their descriptions, ratings etc.
- [**Books:**](https://huggingface.co/datasets/weaviate/agents/viewer/query-agent-books) A dataset that lists books, their authors, descriptions and genres.


## 1. Setting Up Weaviate & Importing Data

To use the Weaviate Query Agent, first, create a [Weaviate Cloud](https://weaviate.io/deployment/serverless?utm_source=webinar&utm_campaign=agents&utm_content=reimagining-data-workflows) account👇
1. [Create Serverless Weaviate Cloud account](https://weaviate.io/deployment/serverless?utm_source=webinar&utm_campaign=agents&utm_content=reimagining-data-workflows) and setup a free [Sandbox](https://weaviate.io/developers/wcs/manage-clusters/create#sandbox-clusters?utm_source=webinar&utm_campaign=agents&utm_content=reimagining-data-workflows)
2. Go to 'Embedding' and enable it, by default, this will make it so that we use `Snowflake/snowflake-arctic-embed-l-v2.0` as the embedding model
3. Take note of the `WEAVIATE_URL` and `WEAVIATE_API_KEY` to connect to your cluster below

> Info: We recommend using [Weaviate Embeddings](https://weaviate.io/developers/weaviate/model-providers/weaviate?utm_source=webinar&utm_campaign=agents&utm_content=reimagining-data-workflows) so you do not have to provide any extra keys for external embedding providers.

In [ ]:
!pip install 'weaviate-client[agents]' datasets

In [3]:
import os
from getpass import getpass

if "WEAVIATE_API_KEY" not in os.environ:
  os.environ["WEAVIATE_API_KEY"] = getpass("Weaviate API Key")
if "WEAVIATE_URL" not in os.environ:
  os.environ["WEAVIATE_URL"] = getpass("Weaviate URL")

In [4]:
import weaviate
from weaviate.auth import Auth

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=os.environ.get("WEAVIATE_URL"),
    auth_credentials=Auth.api_key(os.environ.get("WEAVIATE_API_KEY")),
)

### Prepare the Collections

In the following code blocks, we are pulling our demo datasets from Hugging Face and writing them to new collections in our Weaviate Serverless cluster.

> ❗️ The `QueryAgent` uses the descriptions of collections and properties to decide which ones to use when solving queries, and to access more information about properties. You can experiment with changing these descriptions, providing more detail, and more. It's good practice to provide property descriptions too.

In [ ]:
from weaviate.classes.config import Configure, Property, DataType

# client.collections.delete("Movies")
# client.collections.delete("Books")
client.collections.create(
    "Books",
    description="A dataset that lists books, their author, description and genres",
    vectorizer_config=Configure.Vectorizer.text2vec_weaviate(),
    properties=[
        Property(name="title", data_type=DataType.TEXT, description="title of the book"),
        Property(name="author", data_type=DataType.TEXT, description="author of the book"),
        Property(name="description", data_type=DataType.TEXT, description="description of the book"),
        Property(name="genres", data_type=DataType.TEXT_ARRAY, description="genres of the book"),
      ]
)

client.collections.create(
    "Movies",
    description="A dataset that lists movies, their ratings, original language etc..",
    vectorizer_config=Configure.Vectorizer.text2vec_weaviate(),
    properties=[
        Property(
            name="title",
            data_type=DataType.TEXT,
            description="The title of the movie",
        ),
        Property(
            name="release_date",
            data_type=DataType.TEXT,
            description="The release date of the movie",
        ),
        Property(
            name="overview",
            data_type=DataType.TEXT,
            description="Short description of the movie",
        ),
        Property(
            name="genres",
            data_type=DataType.TEXT_ARRAY,
            description="The genres of the movie",
        ),
        Property(
            name="vote_average",
            data_type=DataType.NUMBER,
            description="vote average of the movie",
        ),
        Property(
            name="vote_count",
            data_type=DataType.INT,
            description="vote count of the movie",
        ),
        Property(
            name="popularity",
            data_type=DataType.NUMBER,
            description="popularity of the movie",
        ),
        Property(
            name="poster_url",
            data_type=DataType.TEXT,
            description="poster path of the movie",
            skip_vectorization=True,
        ),
        Property(
            name="original_language",
            data_type=DataType.TEXT,
            description="Code of the language of the movie",
            skip_vectorization=True,
        ),
    ]
)


In [5]:
from datasets import load_dataset

# movies_dataset = load_dataset("weaviate/agents", "personalization-agent-movies", split="train", streaming=True)
# books_dataset = load_dataset("weaviate/agents", "query-agent-books", split="train", streaming=True)

movies_collection = client.collections.get("Movies")
books_collection = client.collections.get("Books")

# with movies_collection.batch.dynamic() as batch:
#     for item in movies_dataset:
#         batch.add_object(properties=item["properties"])

# with books_collection.batch.dynamic() as batch:
#     for item in books_dataset:
#         batch.add_object(properties=item["properties"])

/Users/tuanacelik/miniconda3/envs/agent/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Set Up the Query Agent

Let's start with a simple agent. Here, we're creating an `agent` that has access to our `Books` & `Movies` datasets.

In [6]:
from weaviate.agents.query import QueryAgent

agent = QueryAgent(
    client=client, collections=["Books", "Movies"],
)

## 3. Run the Query Agent
The `QueryAgent` will determine wither a given query is a regular searcg query (vector search), or whether it requires aggregations, or both.

In [7]:
response = agent.run("What are some good fantasy films that involve elves?")
response.display()

╭─────────────────────────────────────────────── 🔍 Original Query ───────────────────────────────────────────────╮
│                                                                                                                 │
│ What are some good fantasy films that involve elves?                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📝 Final Answer ────────────────────────────────────────────────╮
│                                                                                                                 │
│ Here are some fantasy films that involve elves:                                                                 │
│                                                                                                                 │
│ 1. **Throne of Elves (2016)**: Set in the mythical Elven world of Altera, this animated film features an epic   │
│ battle between Princess Liya and the Dark Elf Elena.                                                            │
│                                                                                                                 │
│ 2. **David and the Elves (2021)**: This film involves a young boy named David and an elf named Albert who       │
│ embark on a journey to rediscover Christmas.                                                                    │
│                                                                                                                 │
│ 3. **Dragon Nest: Warriors' Dawn (2014)**: This animated feature includes elves uniting with humans to fight    │
│ against the awakening of a legendary Black Dragon.                                                              │
│                                                                                                                 │
│ 4. **Onward (2020)**: A suburban fantasy about two elf brothers on a quest to find out if magic still exists.   │
│                                                                                                                 │
│ 5. **A Boy Called Christmas (2021)**: The story follows a young boy named Nikolas who ventures to the North     │
│ Pole to find Elfhelm, the fabled village of elves.                                                              │
│                                                                                                                 │
│ 6. **Alien Xmas (2020)**: An elf mistakes an alien for a toy, not knowing it's intent on disrupting Earth.      │
│                                                                                                                 │
│ 7. **Willow (1988)**: While focusing on a sorcerer, this film involves a magical journey where elves and other  │
│ mythical creatures play a part.                                                                                 │
│                                                                                                                 │
│ 8. **Bright: Samurai Soul (2021)**: The film includes a young elf girl and a journey to the land of the elves.  │
│                                                                                                                 │
│ 9. **Strange Magic (2015)**: A musical fantasy about fairies, elves, and a love potion.                         │
│                                                                                                                 │
│ These films feature a variety of narratives ranging from epic battles and magical quests to heartwarming tales  │
│ of friendship and adventure.                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 1/1 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['fantasy films that involve elves'],                                                               │
│     filters=[[]],                                                                                               │
│     filter_operators='AND',                                                                                     │
│     collection='Movies'                                                                                         │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 📊 No Aggregations Run                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📚 Sources ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  - object_id='d6b48e74-b6f9-4fb0-96c9-109e8b431c83' collection='Movies'                                         │
│  - object_id='58a537df-670e-4d86-9c56-4a6dc9cb3317' collection='Movies'                                         │
│  - object_id='96982f3c-9847-4b76-a63d-9f3bb859fbf2' collection='Movies'                                         │
│  - object_id='85ffcd5f-9ce0-4f99-9f79-08d951bd8c4f' collection='Movies'                                         │
│  - object_id='85b1a40a-dd84-4429-92cf-c64d0a6fa0bf' collection='Movies'                                         │
│  - object_id='8f465d6f-afe1-4aa9-87c5-7405450aeacc' collection='Movies'                                         │
│  - object_id='6ad13de6-7908-4aa2-8234-e55bae0f8692' collection='Movies'                                         │
│  - object_id='3f34ddcc-cf04-4d4b-a1bd-ac6529736495' collection='Movies'                                         │
│  - object_id='8d2f2f51-b058-40d6-b95c-7b8054b33d0f' collection='Movies'                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

   📊 Usage Statistics   
┌────────────────┬──────┐
│ LLM Requests:  │ 3    │
│ Input Tokens:  │ 6414 │
│ Output Tokens: │ 463  │
│ Total Tokens:  │ 6877 │
└────────────────┴──────┘

Total Time Taken: 15.97s

### Ask a follow up question

The agent can also be provided with additional context. For example, we can provide the previous response as context and get a `new_response`

In [8]:
new_response = agent.run("And what are some books with a similar vibe to these films?", context=response)
new_response.display()

╭─────────────────────────────────────────────── 🔍 Original Query ───────────────────────────────────────────────╮
│                                                                                                                 │
│ And what are some books with a similar vibe to these films?                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📝 Final Answer ────────────────────────────────────────────────╮
│                                                                                                                 │
│ Here are some books that share a similar vibe to fantasy films featuring elves, blending elements of fantasy,   │
│ magic, and adventure:                                                                                           │
│                                                                                                                 │
│ 1. **"Lark" by Erica Cope** - This book involves a world of elves and a prophecy, much like many epic fantasy   │
│ films. It includes themes of magic and adventure with a central young protagonist discovering their magical     │
│ heritage.                                                                                                       │
│                                                                                                                 │
│ 2. **"The Dark Elf Trilogy Collector's Edition" by R.A. Salvatore** - This trilogy delves deep into the world   │
│ of dark elves with epic adventure and high fantasy elements, much like the grand narratives in films such as    │
│ the "Lord of the Rings" series.                                                                                 │
│                                                                                                                 │
│ 3. **"The Elf Queen of Shannara" by Terry Brooks** - Part of a larger epic fantasy series, this book revolves   │
│ around the mystical realm of elves and quests, similar to film series like "The Chronicles of Narnia."          │
│                                                                                                                 │
│ 4. **"Blood of Elves" by Andrzej Sapkowski (The Witcher series)** - This story incorporates themes of war,      │
│ prophecy, and the interaction between different races like humans and elves, reminiscent of complex narratives  │
│ seen in films like "The Hobbit."                                                                                │
│                                                                                                                 │
│ 5. **"The Elfstones of Shannara" by Terry Brooks** - Continuing the epic journey in a world filled with ancient │
│ magic and dark forces, catering to fans of fantasy films with rich storytelling and adventure.                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 1/2 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['fantasy books with elves'],                                                                       │
│     filters=[[]],                                                                                               │
│     filter_operators='AND',                                                                                     │
│     collection='Books'                                                                                          │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 2/2 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['books with themes similar to fantasy films'],                                                     │
│     filters=[[]],                                                                                               │
│     filter_operators='AND',                                                                                     │
│     collection='Books'                                                                                          │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 📊 No Aggregations Run                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📚 Sources ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  - object_id='5678eb39-b17e-4ccb-b8c0-a00968ce06c4' collection='Books'                                          │
│  - object_id='37c4b9a1-f19b-412e-b992-5a9d579f3309' collection='Books'                                          │
│  - object_id='cb11b35b-7c90-415d-b41a-bc24d113e1f5' collection='Books'                                          │
│  - object_id='277a86af-4872-479b-adc9-ce4e77bee2d8' collection='Books'                                          │
│  - object_id='af9ad35d-3774-4882-8b0d-b46f8e1f4e6e' collection='Books'                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

   📊 Usage Statistics    
┌────────────────┬───────┐
│ LLM Requests:  │ 5     │
│ Input Tokens:  │ 11653 │
│ Output Tokens: │ 614   │
│ Total Tokens:  │ 12267 │
└────────────────┴───────┘

Total Time Taken: 20.57s

Now let's try a question that sholud require an aggregation. Let's see which author has the most books in our collection.

In [9]:
response = agent.run("Which author has the most books listed in our collection?")
response.display()

╭─────────────────────────────────────────────── 🔍 Original Query ───────────────────────────────────────────────╮
│                                                                                                                 │
│ Which author has the most books listed in our collection?                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📝 Final Answer ────────────────────────────────────────────────╮
│                                                                                                                 │
│ The author with the most books listed in the collection is Stephen King, with 57 books.                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 🔭 No Searches Run                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 📊 Aggregations Run 1/1 ────────────────────────────────────────────╮
│                                                                                                                 │
│ AggregationResultWithCollection(                                                                                │
│     search_query=None,                                                                                          │
│     groupby_property=None,                                                                                      │
│     aggregations=[                                                                                              │
│         TextPropertyAggregation(                                                                                │
│             property_name='author',                                                                             │
│             metrics=<TextMetrics.TOP_OCCURRENCES: 'TOP_OCCURRENCES'>,                                           │
│             top_occurrences_limit=1                                                                             │
│         )                                                                                                       │
│     ],                                                                                                          │
│     filters=[],                                                                                                 │
│     collection='Books'                                                                                          │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

   📊 Usage Statistics   
┌────────────────┬──────┐
│ LLM Requests:  │ 3    │
│ Input Tokens:  │ 3389 │
│ Output Tokens: │ 145  │
│ Total Tokens:  │ 3534 │
└────────────────┴──────┘

Total Time Taken: 6.10s

In [10]:
response = agent.run("And are there any films based on this author's books?", context=response)
response.display()

╭─────────────────────────────────────────────── 🔍 Original Query ───────────────────────────────────────────────╮
│                                                                                                                 │
│ And are there any films based on this author's books?                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📝 Final Answer ────────────────────────────────────────────────╮
│                                                                                                                 │
│ Yes, there are several films based on Stephen King's books. Some notable examples include *Big Driver* (2014),  │
│ which is based on a novella by Stephen King, *Creepshow* (1982), a film that brings five tales of terror to the │
│ screen inspired by E.C. comics of the 1950s, and *Sometimes They Come Back* (1991), based on a short story by   │
│ Stephen King. Additionally, *Cat's Eye* (1985) includes three short stories by Stephen King tied together in a  │
│ triptych format.                                                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 1/1 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=["films based on Stephen King's books"],                                                            │
│     filters=[[]],                                                                                               │
│     filter_operators='AND',                                                                                     │
│     collection='Movies'                                                                                         │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 📊 No Aggregations Run                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📚 Sources ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  - object_id='20b016ae-277e-4b03-a89a-202d6e6917df' collection='Movies'                                         │
│  - object_id='32d5c53f-0768-4cbf-bfc0-18410f72ba29' collection='Movies'                                         │
│  - object_id='88ddd9d3-f326-41a2-9f8a-2ae199f05c4c' collection='Movies'                                         │
│  - object_id='3f0db086-c0d2-4ac6-aa2c-ed1870d4e341' collection='Movies'                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

   📊 Usage Statistics   
┌────────────────┬──────┐
│ LLM Requests:  │ 4    │
│ Input Tokens:  │ 9531 │
│ Output Tokens: │ 370  │
│ Total Tokens:  │ 9901 │
└────────────────┴──────┘

Total Time Taken: 11.39s

### Search over multiple collections

In some cases, we need to combine the results of searches across multiple collections.

In [11]:
response = agent.run("I'm interested in historical fiction books, can you recommend any good ones?"
                     "Are there any films based on historical fiction? And on average, what's the original language that they were filmed in?")

response.display()

╭─────────────────────────────────────────────── 🔍 Original Query ───────────────────────────────────────────────╮
│                                                                                                                 │
│ I'm interested in historical fiction books, can you recommend any good ones?Are there any films based on        │
│ historical fiction? And on average, what's the original language that they were filmed in?                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📝 Final Answer ────────────────────────────────────────────────╮
│                                                                                                                 │
│ Here are some historical fiction books you might enjoy:                                                         │
│                                                                                                                 │
│ 1. "Castles, Customs, and Kings: True Tales by English Historical Fiction Authors" - This compilation includes  │
│ essays that provide historical information from Roman Britain to early 20th-century England.                    │
│ 2. "Post Captain" by Patrick O'Brian - A continuation of the "Master and Commander" series, celebrated for its  │
│ historical accuracy and engaging storytelling.                                                                  │
│ 3. "People of the Book" by Geraldine Brooks - A complex novel following a rare manuscript through centuries of  │
│ war and exile.                                                                                                  │
│ 4. "The Shadow of the Wind" by Carlos Ruiz Zafón - A mysterious novel set in post-war Barcelona, involving a    │
│ hidden library and forgotten books.                                                                             │
│                                                                                                                 │
│ Several films have been made based on historical fiction, including:                                            │
│                                                                                                                 │
│ - "Hamilton" – A film version of the Broadway musical about American founding father Alexander Hamilton.        │
│ - "The Lost City of Z" – Depicts the true-life adventures of British explorer Col. Percy Fawcett in the Amazon. │
│ - "Suffragette" – Based on true events about the early feminist movement.                                       │
│ - "Denial" – The story about a libel case to prove the occurrence of the Holocaust.                             │
│                                                                                                                 │
│ The majority of films based on historical fiction are filmed in English, which is the original language for     │
│ around 75.7% of these films.                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 1/2 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['historical fiction books'],                                                                       │
│     filters=[[]],                                                                                               │
│     filter_operators='AND',                                                                                     │
│     collection='Books'                                                                                          │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 2/2 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['historical fiction films'],                                                                       │
│     filters=[[]],                                                                                               │
│     filter_operators='AND',                                                                                     │
│     collection='Movies'                                                                                         │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 📊 Aggregations Run 1/1 ────────────────────────────────────────────╮
│                                                                                                                 │
│ AggregationResultWithCollection(                                                                                │
│     search_query='historical fiction',                                                                          │
│     groupby_property=None,                                                                                      │
│     aggregations=[                                                                                              │
│         TextPropertyAggregation(                                                                                │
│             property_name='original_language',                                                                  │
│             metrics=<TextMetrics.TOP_OCCURRENCES: 'TOP_OCCURRENCES'>,                                           │
│             top_occurrences_limit=5                                                                             │
│         )                                                                                                       │
│     ],                                                                                                          │
│     filters=[],                                                                                                 │
│     collection='Movies'                                                                                         │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📚 Sources ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  - object_id='b23a88d8-5d49-4871-a69e-31e11761a14c' collection='Books'                                          │
│  - object_id='7789f34a-2b9f-4a9f-82e0-26575faac318' collection='Books'                                          │
│  - object_id='caff6c2d-cef6-4ce9-98f3-48cf57d66b31' collection='Books'                                          │
│  - object_id='81023cdc-1332-4cab-973c-c28c7ccfae73' collection='Books'                                          │
│  - object_id='3462b4a8-da0d-442d-8c52-4691acbcfc5a' collection='Movies'                                         │
│  - object_id='81c3c4e7-b093-447b-8ba0-a443b4ea63a9' collection='Movies'                                         │
│  - object_id='600fa2ea-b0a2-4d87-98bc-d0d27f9f5ad2' collection='Movies'                                         │
│  - object_id='3a29cb86-22d8-4cd4-9c63-1b29ef237987' collection='Movies'                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

   📊 Usage Statistics    
┌────────────────┬───────┐
│ LLM Requests:  │ 5     │
│ Input Tokens:  │ 9640  │
│ Output Tokens: │ 521   │
│ Total Tokens:  │ 10161 │
└────────────────┴───────┘

Total Time Taken: 22.97s

### Changing the System Prompt

In some cases, you may want to define a custom `system_prompt` for your agent. This can help you provide the agent with some default instructions as to how to behave. For example, let's create an agent that is designed to give short bullet point answers.


In [12]:
new_agent = QueryAgent(
    client=client, collections=["Books", "Movies"],
    system_prompt="You are a helpful movies and books assistant that always responds in short, bullet point answsers."
)

For example, this time lets ask something that is about weather!

In [13]:
response = new_agent.run("I'm interested in historical fiction books, can you recommend any good ones?"
                         "Are there any films based on historical fiction? And on average, what's the original language that they were filmed in?")
print(response.final_answer)

- Recommended historical fiction books:
  1. "Castles, Customs, and Kings: True Tales by English Historical Fiction Authors" by Debra Brown
  2. "Post Captain" by Patrick O'Brian
  3. "People of the Book" by Geraldine Brooks
  4. "The Shadow of the Wind" by Carlos Ruiz Zafón
  5. "The Winds of War" by Herman Wouk

- Films based on historical fiction:
  1. "Hamilton" (2020)
  2. "The Lost City of Z" (2017)
  3. "The Kashmir Files" (2022)
  4. "Suffragette" (2015)
  5. "Denial" (2016)
  6. "Hostiles" (2017)

- Average original language of films based on historical fiction is predominantly English.
